<a href="https://colab.research.google.com/github/ErickDiaz17799/ProyectosMaestria/blob/main/Servidores%20y%20APIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install flask flask-ngrok twilio gspread pyngrok
!pip install google-auth google-auth-oauthlib google-auth-httplib2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.1 MB/s eta 0:00:00


In [7]:
import os
from flask import Flask, request
from twilio.twiml.messaging_response import MessagingResponse
import gspread
from google.oauth2.service_account import Credentials
from google.colab import auth, files
import json
import pandas as pd
import subprocess
import threading
import google.auth

In [9]:
# Paso 1: Subir y cargar credenciales (una vez)
if not os.path.exists("credentials.json"):
    print("🔺 Sube tu archivo credentials.json")
    uploaded = files.upload()

🔺 Sube tu archivo credentials.json


Saving credentials.json to credentials.json


In [10]:
# Autenticación con Google
auth.authenticate_user()

creds, _ = google.auth.default()
cliente = gspread.authorize(creds)

SHEET_ID = "1v53Zc_cSI3Q3UoLQKEJqwMrY2kdhPmdsyjZ9uOUT6qU"
hoja = cliente.open_by_key(SHEET_ID).sheet1

datos = hoja.get_all_values()
df = pd.DataFrame(datos[1:], columns=datos[0])
print(df.head())

  ID  DESCRIPCION   MARCA     CATEGORIA     STOCK PRECIO BASICO PRECIO PREMIUM
0  1  REFACCION 1  MARCA 1  CATEGORIA 1  STOCK 1        $100.00        $200.00
1  2  REFACCION 2  MARCA 2  CATEGORIA 2   STOCK 2       $100.00        $200.00
2  3  REFACCION 3  MARCA 3  CATEGORIA 3   STOCK 3       $100.00        $200.00
3  4  REFACCION 4  MARCA 4  CATEGORIA 4   STOCK 4       $100.00        $200.00
4  5  REFACCION 5  MARCA 5  CATEGORIA 5   STOCK 5       $100.00        $200.00


In [11]:
# Función de búsqueda
def buscar_refaccion(nombre_refaccion):
    datos = hoja.get_all_records()
    resultados = [fila for fila in datos if nombre_refaccion.lower() in fila['DESCRIPCION'].lower()]
    if resultados:
        respuesta = "🔍 Refacciones encontradas:\n\n"
        for ref in resultados:
            respuesta += f"🔹 *{ref['DESCRIPCION']}*\n"
            respuesta += f"📌 Marca: {ref['Marca']}\n"
            respuesta += f"📦 Categoría: {ref['Categoría']}\n"
            respuesta += f"📊 Stock disponible: {ref['Stock disponible']}\n"
            respuesta += f"💲 Precio Básico: {ref['PRECIO BASICO']}\n"
            respuesta += f"💎 Precio Premium: {ref['PRECIO PREMIUM']}\n"
        return respuesta
    return "❌ No se encontró la refacción en el inventario."

In [12]:
# Configurar Flask
app = Flask(__name__)

@app.route("/whatsapp", methods=["POST"])
def whatsapp_bot():
    incoming_msg = request.values.get("Body", "").strip()
    sender = request.values.get("From", "")
    print(f"📩 Mensaje recibido de {sender}: {incoming_msg}")

    resp = MessagingResponse()

    if incoming_msg:
        respuesta = buscar_refaccion(incoming_msg)
        resp.message(respuesta)
    else:
        resp.message("❌ No entendí tu mensaje. Escribe el nombre de la refacción.")

    return str(resp)

In [13]:
# Ejecutar servidor Flask
def iniciar_servidor():
    print("🚀 Servidor Flask iniciado en Colab")
    app.run()

In [14]:
# Exponer con LocalTunnel
def exponer_servidor():
    print("🌍 Exponiendo el servidor con LocalTunnel...")

    process = subprocess.Popen(["npx", "localtunnel", "--port", "5000"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Leer la salida en tiempo real hasta encontrar la URL
    for line in process.stdout:
        if "your url is:" in line:
            start = line.find("https://")
            public_url = line[start:].strip()
            print(f"🌍 URL pública para Twilio: {public_url}")
            break

In [15]:
exponer_servidor()

🌍 Exponiendo el servidor con LocalTunnel...
🌍 URL pública para Twilio: https://empty-pants-stay.loca.lt


In [ ]:
#!fuser -k 5000/tcp

In [ ]:
import time
import threading

if __name__ == "__main__":
    flask_thread = threading.Thread(target=iniciar_servidor)
    flask_thread.start()

    exponer_servidor()

    # 🚨 Esto es lo que mantiene vivo el kernel:
    while True:
        time.sleep(1)

🚀 Servidor Flask iniciado en Colab🌍 Exponiendo el servidor con LocalTunnel...

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


🌍 URL pública para Twilio: https://real-pens-sing.loca.lt
